In [ ]:
!pip install easyocr
!pip install fitz
!pip install PyMuPDF

In [ ]:
# Convert pdf to images andf removing white spaces fromthe border
def pdf_2_img(pdf_file, img_dir):
    # img_dir.mkdir(exist_ok= True, parents = True)
    doc = fitz.open(pdf_file)
    for page in doc:
        img_file = img_dir + f"page_{page.number}.png"
        #Removing the surrounding white space to make OCR life easier
        rect = page.rect
        tl = rect.top_left + (60,60)
        br = rect.bottom_right - (60,60)
        clip = fitz.Rect(tl,br)  # the area we want
        pix = page.get_pixmap(dpi = 300, clip=clip)
        pix.save(img_file)

In [ ]:
# Cut the image into half vertically creating 2 different images
import cv2
img_dir = os.listdir("/content/drive/MyDrive/OCR/Pdf2Img/")
for i in img_dir[1:]:
  img = cv2.imread("/content/drive/MyDrive/OCR/Pdf2Img/"+str(i))
  height = img.shape[0]
  width = img.shape[1]

  # Cut the image in half
  width_cutoff = width // 2
  s1 = img[:, :width_cutoff]
  s2 = img[:, width_cutoff:]

  cv2.imwrite("/content/drive/MyDrive/OCR/ImgHalves/"+i+"1st.png", s1)
  cv2.imwrite("/content/drive/MyDrive/OCR/ImgHalves/"+i+"2nd.png", s2)


In [ ]:
#Convert image halves to text using easyocr
import easyocr
page_lst = []
c=0
img_half_dir = os.listdir("/content/drive/MyDrive/OCR/ImgHalves/")
for i in img_half_dir:
  s=''
  reader = easyocr.Reader(['hi'])
  result = reader.readtext("/content/drive/MyDrive/OCR/ImgHalves/"+str(i))  
  result = result[3:]
  for i in range(len(result)):
    s+= " "+result[i][1]
  c+=1
  print('index',c)
  page_lst.append(s.strip())

In [ ]:
#Convert into txt files and arrange them
from docx import Document
import collections
for i in range(0,len(page_lst),2):
  with open(f'/content/sample_data/demo/{i}.txt','w') as file:
    file.write(page_lst[i]+"\n")
    file.write(page_lst[i+1])

txt_file_dir = os.listdir("/content/sample_data/txt_files/")
d={}
for i in txt_file_dir:
  d[i]=int(i.split('.')[0])
sorted_dict = collections.OrderedDict({k: v for k, v in sorted(d.items(), key=lambda item: item[1])})

In [ ]:
#Convert text files into a doc
document = Document()
for i in sorted_dict.keys():
  with open(f'/content/sample_data/txt_files/'+str(i),'r') as file:
    for line in file.readlines():
      document.add_paragraph(line)
    document.add_page_break()
document.save('lsd_01_01_01-07-1952_hindi.docx')